In [5]:
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
pokemon_data = pd.read_csv(file_name)

features = ['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed', 'type1', 'type2']
X = pokemon_data[features]
y = pokemon_data['total']

X['type2'].fillna('None', inplace=True)
X_encoded = pd.get_dummies(X, columns=['type1', 'type2'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

initial_model = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
initial_model.fit(X_train, y_train)

initial_train_accuracy = initial_model.score(X_train, y_train)
initial_test_accuracy = initial_model.score(X_test, y_test)

print("Initial Train Accuracy:", initial_train_accuracy)
print("Initial Test Accuracy:", initial_test_accuracy)

param_grid = {
    'hidden_layer_sizes': [(50, 50), (100, 50), (100, 100)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant', 'adaptive'],
}
grid_search = GridSearchCV(MLPRegressor(max_iter=1000, random_state=42), param_grid, n_jobs=-1, cv=3)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

pokemon_records = []

while True:
    print("\n포켓몬의 능력치를 입력하세요:")
    hp = float(input("HP: "))
    attack = float(input("Attack: "))
    defense = float(input("Defense: "))
    sp_atk = float(input("Sp. Attack: "))
    sp_def = float(input("Sp. Def: "))
    speed = float(input("Speed: "))
    type_1 = input("Type 1: ")
    type_2 = input("Type 2 (없으면 Enter): ")

    new_pokemon = pd.DataFrame([[hp, attack, defense, sp_atk, sp_def, speed, type_1, type_2]],
                               columns=['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed', 'type1', 'type2'])

    new_pokemon['total'] = new_pokemon[['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed']].sum(axis=1)

    if not type_2:
        new_pokemon['type2'] = 'None'

    new_pokemon_encoded = pd.get_dummies(new_pokemon[['type1', 'type2']], columns=['type1', 'type2'])

    new_features = ['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed']
    new_X = new_pokemon[new_features]
    new_X_encoded = pd.concat([new_X, new_pokemon_encoded], axis=1)

    missing_columns = set(X_encoded.columns) - set(new_X_encoded.columns)
    for col in missing_columns:
        new_X_encoded[col] = 0

    new_X_encoded = new_X_encoded[X_encoded.columns]  # Ensure the order of columns is the same as training data

    new_X_scaled = scaler.transform(new_X_encoded)  # Use the existing scaler to transform

    predicted_total = best_model.predict(new_X_scaled)  # Use the best model for prediction

    print("예측된 전투력:", predicted_total[0])

    pokemon_record = {
        'hp': hp,
        'attack': attack,
        'defense': defense,
        'sp_attack': sp_atk,
        'sp_defense': sp_def,
        'speed': speed,
        'type1': type_1,
        'type2': type_2 if type_2 else 'None',
        'total': predicted_total[0]
    }
    pokemon_records.append(pokemon_record)

    continue_input = input("계속 입력하시겠습니까? (Y/N): ")
    if continue_input.upper() != 'Y':
        break

pokemon_records_sorted = sorted(pokemon_records, key=lambda x: x['total'], reverse=True)

print("\n포켓몬 전투력 순위:")
for i, record in enumerate(pokemon_records_sorted, start=1):
    print(f"{i}. {record['type1']} {record['type2'] if record['type2'] != 'None' else ''} - 전투력: {record['total']}")


Saving pokemon.csv.csv to pokemon.csv (3).csv


<ipython-input-5-d310b57b0880>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['type2'].fillna('None', inplace=True)


Initial Train Accuracy: 0.9996107763655893
Initial Test Accuracy: 0.9795389267389452


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1000) reached and the optimization hasn't converged yet.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

uploaded = files.upload()

file_name = list(uploaded.keys())[0]

pokemon_data = pd.read_csv(file_name)

features = ['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed', 'type1', 'type2', 'total']

X = pokemon_data[features]
y = pokemon_data['total']

X_encoded = pd.get_dummies(X, columns=['type1', 'type2'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
model.fit(X_train, y_train)

train_accuracy = accuracy_score(y_train, model.predict(X_train))
test_accuracy = accuracy_score(y_test, model.predict(X_test))

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

while True:
    print("포켓몬의 능력치를 입력하세요:")
    hp = float(input("HP: "))
    attack = float(input("Attack: "))
    defense = float(input("Defense: "))
    sp_atk = float(input("Sp. Attack: "))
    sp_def = float(input("Sp. Def: "))
    speed = float(input("Speed: "))
    type_1 = input("Type 1: ")
    type_2 = input("Type 2 (없으면 Enter): ")

    new_pokemon = pd.DataFrame([[hp, attack, defense, sp_atk, sp_def, speed, type_1, type_2]],
                               columns=['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed', 'type1', 'type2'])

    new_pokemon['total'] = new_pokemon['hp'] + new_pokemon['attack'] + new_pokemon['defense'] + new_pokemon['sp_attack'] + new_pokemon['sp_defense'] + new_pokemon['speed']

    if not type_2:
        new_pokemon['type2'] = 'None'
    new_pokemon_encoded = pd.get_dummies(new_pokemon[['type1', 'type2']], columns=['type1', 'type2'])

    new_features = ['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed']
    new_X = new_pokemon[new_features]
    new_X_encoded = pd.concat([new_X, new_pokemon_encoded], axis=1)

    missing_columns = set(X_encoded.columns) - set(new_X_encoded.columns)
    for col in missing_columns:
        new_X_encoded[col] = 0

    new_scaler = StandardScaler()

    new_X_scaled = new_scaler.fit_transform(new_X_encoded)

    predicted_total = model.predict(new_X_scaled)

    print("예측된 전투력:", predicted_total)
    continue_input = input("계속 입력하시겠습니까? (Y/N): ")
    if continue_input.upper() != 'Y':
        break


In [ ]:
from google.colab import files
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

uploaded = files.upload()

file_name = list(uploaded.keys())[0]

pokemon_data = pd.read_csv(file_name)

features = ['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed', 'type1', 'type2']

X = pokemon_data[features]
y = pokemon_data['total']

X['total'] = X[['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed']].sum(axis=1)


X['type2'].fillna('None', inplace=True)

X_encoded = pd.get_dummies(X, columns=['type1', 'type2'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

model = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)
model.fit(X_train, y_train)

train_accuracy = accuracy_score(y_train, model.predict(X_train))
test_accuracy = accuracy_score(y_test, model.predict(X_test))

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)

pokemon_records = []

while True:
    print("\n포켓몬의 능력치를 입력하세요:")
    hp = float(input("HP: "))
    attack = float(input("Attack: "))
    defense = float(input("Defense: "))
    sp_atk = float(input("Sp. Attack: "))
    sp_def = float(input("Sp. Def: "))
    speed = float(input("Speed: "))
    type_1 = input("Type 1: ")
    type_2 = input("Type 2 (없으면 Enter): ")


    new_pokemon = pd.DataFrame([[hp, attack, defense, sp_atk, sp_def, speed, type_1, type_2]],
                               columns=['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed', 'type1', 'type2'])


    new_pokemon['total'] = new_pokemon[['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed']].sum(axis=1)


    if not type_2:
        new_pokemon['type2'] = 'None'


    new_pokemon_encoded = pd.get_dummies(new_pokemon[['type1', 'type2']], columns=['type1', 'type2'])

    new_features = ['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed']
    new_X = new_pokemon[new_features]
    new_X_encoded = pd.concat([new_X, new_pokemon_encoded], axis=1)

    missing_columns = set(X_encoded.columns) - set(new_X_encoded.columns)
    for col in missing_columns:
        new_X_encoded[col] = 0

    new_scaler = StandardScaler()

    new_X_scaled = new_scaler.fit_transform(new_X_encoded)

    predicted_total = model.predict(new_X_scaled)

    print("예측된 전투력:", predicted_total)


    pokemon_record = {
        'hp': hp,
        'attack': attack,
        'defense': defense,
        'sp_attack': sp_atk,
        'sp_defense': sp_def,
        'speed': speed,
        'type1': type_1,
        'type2': type_2 if type_2 else 'None',
        'total': predicted_total[0]
    }
    pokemon_records.append(pokemon_record)

    continue_input = input("계속 입력하시겠습니까? (Y/N): ")
    if continue_input.upper() != 'Y':
        break

pokemon_records_sorted = sorted(pokemon_records, key=lambda x: x['total'], reverse=True)

print("\n포켓몬 전투력 순위:")
for i, record in enumerate(pokemon_records_sorted, start=1):
    print(f"{i}. {record['type1']} {record['type2'] if record['type2'] != 'None' else ''} - 전투력: {record['total']}")
